# Simple non-interacting harmonic oscillator

In [45]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm

In [46]:
sns.set(color_codes=True)

In [50]:
hbar = 1

In [51]:
class SimpleGaussian:

    def __init__(self, num_particles, num_dimensions, spread, alpha, mass=1, omega=1):
        self.num_particles = num_particles
        self.num_dimensions = num_dimensions
        self.mass = mass
        self.omega = omega
        self.alpha = alpha

        self.particles = spread*(2*np.random.random((num_particles, num_dimensions)) - 1)

    def compute_laplacian(self):
        laplacian = -2*self.num_dimensions*self.num_particles*self.alpha \
            + 4*self.alpha**2*np.sum(self.particles**2)

        return laplacian

    def evaluate(self):
        return np.exp(-self.alpha*np.sum(self.particles**2))

In [52]:
class HarmonicOscillator:

    def __init__(self, mass, omega):
        self.mass = mass
        self.omega = omega

    def compute_local_energy(self, wavefunction):
        kinetic_energy = self.compute_kinetic_energy(wavefunction)
        potential_energy = self.compute_potential_energy(wavefunction)

        return kinetic_energy + potential_energy

    def compute_kinetic_energy(self, wavefunction):
        return - hbar**2*wavefunction.compute_laplacian()/(2.0*self.mass)

    def compute_potential_energy(self, wavefunction):
        return 0.5*self.mass*self.omega**2*np.sum(wavefunction.particles)

In [53]:
def step(wavefunction, step_length):
    prev = wavefunction.evaluate()
    p_index = np.random.randint(wavefunction.num_particles)
    prev_pos = wavefunction.particles[p_index]

    step = step_length*(2*np.random.random(wavefunction.num_dimensions) - 1.0)
    wavefunction.particles[p_index] += step

    if wavefunction.evaluate()**2/prev**2 > np.random.random():
        return True
    else:
        wavefunction.particles[p_index] = prev_pos

    return False

In [54]:
def run(wavefunction, hamiltonian, step_length, num_samples):

    energy = 0
    energy_squared = 0
    num_accepted_states = 0

    local_energy = hamiltonian.compute_local_energy(wavefunction)

    for i in range(num_samples):

        if step(wavefunction, step_length):
            local_energy = hamiltonian.compute_local_energy(wavefunction)
            num_accepted_states += 1

        energy += local_energy
        energy_squared += local_energy**2

    energy /= num_samples
    energy_squared /= num_samples
    variance = energy_squared - energy**2

    return energy, energy_squared, variance, num_accepted_states

In [80]:
mass = omega = 1
spread = 1.0
step_length = 0.1

num_particles = 300
num_dimensions = 1
num_samples = 300000

alpha_start = 0.3
alpha_end = 0.7
num_alphas = 17

alphas = np.linspace(alpha_start, alpha_end, num_alphas)

wavefunction = SimpleGaussian(
    num_particles, num_dimensions, spread, alpha_start, mass=mass, omega=omega)
hamiltonian = HarmonicOscillator(mass, omega)

In [81]:
ret_vals = []
for i in tqdm.tqdm(range(num_alphas)):
    ret_vals.append(run(wavefunction, hamiltonian, step_length, num_samples))

  6%|▌         | 1/17 [00:15<04:00, 15.03s/it]/Users/Schoyen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/Schoyen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
100%|██████████| 17/17 [03:48<00:00, 13.44s/it]


In [88]:
print (ret_vals)

[[ -1.84607853e+01   2.39840627e+03   2.05760567e+03   2.90796000e+05]
 [ -1.11486886e+02   1.24305997e+04   1.27386358e+00   2.39960000e+04]
 [ -2.86101845e+02   8.18542658e+04  -3.25846486e-07   0.00000000e+00]
 [ -4.34821755e+02   1.89069959e+05  -7.87491444e-07   0.00000000e+00]
 [ -6.19182694e+02   3.83387208e+05   3.08791641e-07   0.00000000e+00]
 [ -7.96993881e+02   6.35199246e+05  -1.90641731e-06   0.00000000e+00]
 [ -1.06452981e+03   1.13322372e+06  -1.80676579e-07   0.00000000e+00]
 [ -1.27150665e+03   1.61672915e+06  -1.63679942e-07   0.00000000e+00]
 [ -1.48229899e+03   2.19721029e+06  -1.84569508e-05   0.00000000e+00]
 [ -1.71231585e+03   2.93202557e+06  -2.52639875e-05   0.00000000e+00]
 [ -1.86125447e+03   3.46426819e+06  -2.63201073e-05   0.00000000e+00]
 [ -1.97375689e+03   3.89571625e+06   9.40542668e-06   0.00000000e+00]
 [ -2.12419973e+03   4.51222447e+06   3.07746232e-05   0.00000000e+00]
 [ -2.46599219e+03   6.08111747e+06   3.89749184e-05   0.00000000e+00]
 [ -2.

In [89]:
ret_vals = np.array(ret_vals)
energies = ret_vales[:, 0]

In [90]:
print (energies)

[ 80.40411765  75.99715753  68.96042729  66.02399791  65.0343069
  66.39903676  61.02769663  51.67983474  43.93101121  40.48736888
  31.99313783  22.43303955  17.6513677   12.48616592   7.25193777
   1.54772533  -1.1103213 ]
